In [ ]:
pip install pandas torch torchvision scikit-learn sentence-transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
# Load the data
file_path = "sve_with_summaries.csv"  # Update with your file path
data = pd.read_csv(file_path, delimiter=";", on_bad_lines='skip')

# Extract the relevant columns
texts = data["Summary"].astype(str)
labels = data["područje_rada"].astype(str)

# Encode labels (convert string classes to integers)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    texts, encoded_labels, test_size=0.2, random_state=42
)


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# Load the embedding model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate embeddings for the training and test data
test_embeddings = torch.tensor(embedder.encode(X_test.reset_index(drop=True).tolist(), convert_to_tensor=True))
train_embeddings = torch.tensor(embedder.encode(X_train.reset_index(drop=True).tolist(), convert_to_tensor=True))

<ipython-input-4-03ef148a7716>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_embeddings = torch.tensor(embedder.encode(X_test.reset_index(drop=True).tolist(), convert_to_tensor=True))
<ipython-input-4-03ef148a7716>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_embeddings = torch.tensor(embedder.encode(X_train.reset_index(drop=True).tolist(), convert_to_tensor=True))


In [ ]:
import torch.nn as nn
import torch.optim as optim

class TEXTCLASSIFIER(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TEXTCLASSIFIER, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Hyperparameters
input_dim = train_embeddings.shape[1]
hidden_dim = 128
num_classes = len(label_encoder.classes_)

# Initialize the model, loss, and optimizer
model = TEXTCLASSIFIER(input_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Convert labels to tensors
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

# Training loop
epochs = 20
batch_size = 48

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # Train in mini-batches
    for i in range(0, len(train_embeddings), batch_size):
        batch_embeddings = train_embeddings[i:i+batch_size]
        batch_labels = y_train_tensor[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_embeddings):.4f}")


Epoch 1/20, Loss: 0.0373
Epoch 2/20, Loss: 0.0267
Epoch 3/20, Loss: 0.0234
Epoch 4/20, Loss: 0.0219
Epoch 5/20, Loss: 0.0211
Epoch 6/20, Loss: 0.0205
Epoch 7/20, Loss: 0.0201
Epoch 8/20, Loss: 0.0197
Epoch 9/20, Loss: 0.0194
Epoch 10/20, Loss: 0.0191
Epoch 11/20, Loss: 0.0189
Epoch 12/20, Loss: 0.0186
Epoch 13/20, Loss: 0.0184
Epoch 14/20, Loss: 0.0182
Epoch 15/20, Loss: 0.0181
Epoch 16/20, Loss: 0.0179
Epoch 17/20, Loss: 0.0177
Epoch 18/20, Loss: 0.0175
Epoch 19/20, Loss: 0.0174
Epoch 20/20, Loss: 0.0172


In [ ]:
from sklearn.metrics import classification_report

model.eval()
with torch.no_grad():
    predictions = model(test_embeddings)
    predicted_classes = torch.argmax(predictions, dim=1)

# Decode predicted labels back to class names
predicted_labels = label_encoder.inverse_transform(predicted_classes.numpy())
true_labels = label_encoder.inverse_transform(y_test)

# Print a classification report
print(classification_report(true_labels, predicted_labels))


                                                                                                                                                                   precision    recall  f1-score   support

><span class=""texatom"" id=""MathJax-Span-126""><span class=""mrow"" id=""MathJax-Span-127""><span class=""mi"" id=""MathJax-Span-128"" style=""font-size: 70.7%       0.00      0.00      0.00         1
                                                                                                                                          biomedicina i zdravstvo       0.75      0.74      0.75       408
                                                                                                                                             biotehničke znanosti       0.71      0.57      0.63       413
                                                                                                                                               društvene znanosti       0.59      0.54     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
new_text = ["Cilj ovog rada bio je odrediti optimalne parametre ekstrakcije potpomognute ultrazvukom (UAE) za izolaciju fenolnih spojeva iz kore banane (Musa spp.) te odrediti antioksidacijsku aktivnost ekstrakata. Ekstrakcija je provedena uz 30 %-tni etanol kao otapalo te su ispitani utjecaji omjera mase uzorka i volumena otapala (1:40, 1:50 i 1:60 g/mL), amplitude ultrazvuka (50, 75 i 100 %) i vremena ekstrakcije (5, 10 i 15 min). Maseni udjeli izoliranih spojeva određeni su spektrofotometrijski, a dobivene prosječne vrijednosti iznosile su za hidroksicimetne kiseline 140,02 mg CAE/100 g i flavonoide 58,35 mg QE/100 g. Antioksidacijska aktivnost određena je FRAP metodom s prosječnom vrijednošću 5,09 mmol TE/100 g i ABTS metodom 5,44 mmol TE/100 g"]

new_embedding = torch.tensor(embedder.encode(new_text, convert_to_tensor=True))

model.eval()
with torch.no_grad():
    new_prediction = model(new_embedding)
    predicted_class = torch.argmax(new_prediction).item()

predicted_label = label_encoder.inverse_transform([predicted_class])
print(f"Predicted Class: {predicted_label[0]}")


Predicted Class: biotehničke znanosti


<ipython-input-8-090da05b3bed>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_embedding = torch.tensor(embedder.encode(new_text, convert_to_tensor=True))


In [ ]:
model_scripted = torch.jit.script(model)

model_scripted.save('./model_scripted.pt')

torch.save(label_encoder, './label_encoder.lblenc')

torch.save(embedder, './embedder.emb')

In [ ]:
import torch
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

model =  torch.jit.load( "./model_scripted.pt")
#model.eval()
label_encoder = torch.load("./label_encoder.lblenc")
embedder = torch.load("./embedder.emb")
#embedder.eval()

In [ ]:
new_text = ["Cilj ovog rada bio je odrediti optimalne parametre ekstrakcije potpomognute ultrazvukom (UAE) za izolaciju fenolnih spojeva iz kore banane (Musa spp.) te odrediti antioksidacijsku aktivnost ekstrakata. Ekstrakcija je provedena uz 30 %-tni etanol kao otapalo te su ispitani utjecaji omjera mase uzorka i volumena otapala (1:40, 1:50 i 1:60 g/mL), amplitude ultrazvuka (50, 75 i 100 %) i vremena ekstrakcije (5, 10 i 15 min). Maseni udjeli izoliranih spojeva određeni su spektrofotometrijski, a dobivene prosječne vrijednosti iznosile su za hidroksicimetne kiseline 140,02 mg CAE/100 g i flavonoide 58,35 mg QE/100 g. Antioksidacijska aktivnost određena je FRAP metodom s prosječnom vrijednošću 5,09 mmol TE/100 g i ABTS metodom 5,44 mmol TE/100 g"]

new_embedding = torch.tensor(embedder.encode(new_text, convert_to_tensor=True))

model.eval()
with torch.no_grad():
    new_prediction = model(new_embedding)
    probabilities = F.softmax(new_prediction, dim=1)
    confidence = torch.max(probabilities).item()

    predicted_class = torch.argmax(new_prediction).item()
    predicted_label = label_encoder.inverse_transform([predicted_class])

predicted_label = label_encoder.inverse_transform([predicted_class])
print(f"Predicted Class: {predicted_label[0]}")
print(f"Confidence: {confidence:.4f}")

Predicted Class: biotehničke znanosti
Confidence: 0.8951
